<a href="https://colab.research.google.com/github/AlejoBI/IO/blob/main/Taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
from ortools.linear_solver import pywraplp

# Crear el solucionador
solucionador = 'SCIP'
solver = pywraplp.Solver.CreateSolver(solucionador)

In [ ]:
# Parámetros
dias = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']  # Días de la semana
turnos = ['mañana', 'tarde', 'noche']  # Turnos
bodegas = ['Bodega 1', 'Bodega 2']  # Nombres de bodegas
operarios_b1 = ['Cristian', 'Yuliana', 'Jose Daniel', 'Kevin']  # Operarios en Bodega 1
operarios_b2 = ['Daniela', 'Alison', 'Sebastián', 'Alex']  # Operarios en Bodega 2

# Definir variables de decisión
X = {}
for o in range(4):  # Para cada operario
    for b in range(len(bodegas)):  # Para cada bodega (índice)
        for d in range(len(dias)):  # Para cada día (índice)
            for t in range(len(turnos)):  # Para cada turno (índice)
                X[o, b, d, t] = solver.IntVar(0, 1, f'X_{o}{b}{d}_{t}')

# Restricciones

# R1: Cada operario solo puede trabajar un turno por día en su respectiva bodega
for o in range(4):
    for b in range(len(bodegas)):
        for d in range(len(dias)):
            solver.Add(sum(X[o, b, d, t] for t in range(len(turnos))) <= 1)

# R2: Solo un operario por turno en cada bodega y día
for b in range(len(bodegas)):  # Para cada bodega
    for d in range(len(dias)):  # Para cada día
        for t in range(len(turnos)):  # Para cada turno
            solver.Add(sum(X[o, b, d, t] for o in range(4)) <= 1)

# R3: Actividades de inventario solo en la mañana y tarde el último día (domingo)
for b in range(len(bodegas)):
    for o in range(4):
        solver.Add(X[o, b, 6, 2] == 0)  # Sin turno nocturno el domingo

# R4 y R5: Los operarios aprendices no pueden trabajar ni el segundo ni el cuarto día en los turnos de noche
solver.Add(sum(X[0, 0, d, 2] for d in [1, 3]) == 0)  # Cristian en Bodega 1
solver.Add(sum(X[1, 1, d, 2] for d in [1, 3]) == 0)  # Daniela en Bodega 2

# R6: Mínimo 20 turnos a la semana en total por bodega
for b in range(len(bodegas)):
    solver.Add(sum(X[o, b, d, t] for o in range(4) for d in range(len(dias)) for t in range(len(turnos))) >= 20)

# R7: Ningún operario trabaja más de 2 turnos nocturnos en la semana
for o in range(4):
    for b in range(len(bodegas)):
        solver.Add(sum(X[o, b, d, 2] for d in range(len(dias))) <= 2)  # Turno nocturno es el índice 2

# R8: Todos los operarios trabajan 5 turnos por semana en su respectiva bodega (uniforme)
for o in range(4):
    for b in range(len(bodegas)):
        solver.Add(sum(X[o, b, d, t] for d in range(len(dias)) for t in range(len(turnos))) == 5)

# R9: Turno diurno solo si no hay nocturno el día anterior
for o in range(4):
    for b in range(len(bodegas)):
        for d in range(1, len(dias)):  # Comenzar desde Martes (índice 1)
            solver.Add(X[o, b, d, 0] <= 1 - X[o, b, d-1, 2])  # Turno diurno solo si no hay nocturno el día anterior

# R10, R11 y R12: Solicitudes específicas de los operarios
solver.Add(X[1, 0, 1, 1] == 0)  # Yuliana no trabaja el turno de tarde los martes
solver.Add(X[2, 1, 4, 2] == 1)  # Sebastián trabaja el turno de noche los viernes
solver.Add(X[1, 1, 0, 0] == 0)  # Alison no trabaja el turno de mañana los lunes


# Invocar solución (no necesitamos función objetivo, solo verificar factibilidad)
status = solver.Solve()

# Revisar el estado de la solución
if status == pywraplp.Solver.OPTIMAL:
    print('Los turnos de cada operario según las políticas y necesidades de la empresa para la semana son: ')
    for o in range(4):
        for b in range(len(bodegas)):
            for d in range(len(dias)):
                for t in range(len(turnos)):
                    if X[o, b, d, t].solution_value() == 1:
                        operario = operarios_b1[o] if b == 0 else operarios_b2[o]
                        print(f"Operario/a {operario} trabaja el día {dias[d]} en la {bodegas[b]} en el turno de la {turnos[t]}")
else:
    print('No se encontró una solución factible.')

Los turnos de cada operario según las políticas y necesidades de la empresa para la semana son: 
Operario/a Cristian trabaja el día lunes en la Bodega 1 en el turno de la tarde
Operario/a Cristian trabaja el día martes en la Bodega 1 en el turno de la tarde
Operario/a Cristian trabaja el día miércoles en la Bodega 1 en el turno de la mañana
Operario/a Cristian trabaja el día viernes en la Bodega 1 en el turno de la tarde
Operario/a Cristian trabaja el día sábado en la Bodega 1 en el turno de la noche
Operario/a Daniela trabaja el día lunes en la Bodega 2 en el turno de la noche
Operario/a Daniela trabaja el día miércoles en la Bodega 2 en el turno de la mañana
Operario/a Daniela trabaja el día jueves en la Bodega 2 en el turno de la mañana
Operario/a Daniela trabaja el día viernes en la Bodega 2 en el turno de la mañana
Operario/a Daniela trabaja el día sábado en la Bodega 2 en el turno de la mañana
Operario/a Yuliana trabaja el día lunes en la Bodega 1 en el turno de la noche
Operario